<a href="https://colab.research.google.com/github/SheikhMudassarHanif/NLP/blob/main/21F9176_21F9342_9A_Assignment3NLPFINETUNNINGLLMS_TEXTSUMMARIZATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer



#BART

#Using BART (BERT MODEL FOR SUMMARIZATION) and its tokeinzer (pretrained)  
#PYTORCH

In [ ]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")


#Loading the dataset for Summarization

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df=pd.read_csv('/content/samsum-test.csv')
df=df[['dialogue','summary']]
df.head()

,dialogue,summary
0,"Hannah: Hey, do you have Betty's number?\nAman...",Hannah needs Betty's number but Amanda doesn't...
1,Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric:...,Eric and Rob are going to watch a stand-up on ...
2,"Lenny: Babe, can you help me with something?\r...",Lenny can't decide which trousers to buy. Bob ...
3,"Will: hey babe, what do you want for dinner to...",Emma will be home soon and she will let Will k...
4,"Ollie: Hi , are you in Warsaw\r\nJane: yes, ju...",Jane is in Warsaw. Ollie and Jane has a party....


Separating inputs and labels

In [ ]:
inputs=df['dialogue'].tolist()
labels=df['summary'].tolist()

In [ ]:
from sklearn.model_selection import train_test_split
train_inputs,val_inputs,train_labels,val_labels=train_test_split(inputs,labels,test_size=0.2,random_state=42)

In [ ]:
len(train_inputs) , len(val_inputs) , len(train_labels) , len(val_labels)

(655, 164, 655, 164)

#now applying pretrained bert tokeinzer on labels and inputs

In [ ]:
def tokenize(inputs, labels,tokenizer):
    model_input = tokenizer(inputs, max_length=512, padding='max_length', truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(labels, max_length=60, padding='max_length', truncation=True)

    model_input['labels'] = labels['input_ids']
    return model_input


In [ ]:
train_tokenized_data=tokenize(train_inputs,train_labels,tokenizer)
val_tokenized_data=tokenize(val_inputs,val_labels,tokenizer)



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_dict(train_tokenized_data)
val_dataset = Dataset.from_dict(val_tokenized_data)


In [ ]:
print("Input IDs:", train_tokenized_data['input_ids'][:5])  # Print first 5 for verification
print("Labels:", train_tokenized_data['labels'][:5])
print("Attention Mask:", train_tokenized_data['attention_mask'][:5])  # This should be printed


Input IDs: [[0, 15691, 35, 3394, 18, 198, 116, 50121, 50118, 20907, 35, 2486, 38594, 328, 50121, 50118, 15691, 35, 2381, 26536, 2512, 2722, 50121, 50118, 25244, 35, 38, 437, 45, 11, 1139, 6, 6661, 12563, 328, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir='./results2',
    evaluation_strategy="epoch",  # Evaluate after every epoch
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=10,  # Log every 10 steps
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # Ensure you provide this for evaluation
    tokenizer=tokenizer,
)


<ipython-input-21-5777441c4ecf>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()



Epoch,Training Loss,Validation Loss
1,0.820000,0.831584
2,0.623800,0.832831
3,0.596600,0.838039
4,0.472700,0.843954
5,0.436200,0.870724
6,0.372200,0.906437
7,0.365900,0.903236
8,0.336500,0.926090
9,0.296500,0.933227
10,0.283400,0.937665


TrainOutput(global_step=410, training_loss=0.46772447853553584, metrics={'train_runtime': 495.1711, 'train_samples_per_second': 13.228, 'train_steps_per_second': 0.828, 'total_flos': 1996886900736000.0, 'train_loss': 0.46772447853553584, 'epoch': 10.0})

In [ ]:
model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [ ]:
import torch

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Path to the directory where your model is saved
model_directory = "/content/results2/checkpoint-410"  # Update this with your directory path

# Load the saved model and tokenizer from the directory
model = AutoModelForSeq2SeqLM.from_pretrained(model_directory)
# tokenizer = AutoTokenizer.from_pretrained(model_directory)

# Move the model to the same device (GPU/CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [ ]:
# Assuming your model and tokenizer are already loaded

# Text you want to summarize
text = """Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park
together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him
Hannah: Urgh.. Alright
Hannah: Bye"""

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Move the input tensors to the same device as the model
inputs = {key: value.to(device) for key, value in inputs.items()}

# Generate the summary
with torch.no_grad():
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=150, length_penalty=2.0, early_stopping=True)

# Decode the generated summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the summary
print("Generated Summary:", summary)


Generated Summary: Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.


In [ ]:
import evaluate

# Load the Rouge metric
rouge = evaluate.load("rouge")
reference_summary=text
generated_summary=summary
results = rouge.compute(predictions=[generated_summary], references=[reference_summary])
print("Rouge Score:", results)
print("ROUGE-1 F1 Score:", results["rouge1"])
print("ROUGE-2 F1 Score:", results["rouge2"])
print("ROUGE-L F1 Score:", results["rougeL"])

Rouge Score: {'rouge1': 0.19780219780219782, 'rouge2': 0.0449438202247191, 'rougeL': 0.17582417582417584, 'rougeLsum': 0.17582417582417584}
ROUGE-1 F1 Score: 0.19780219780219782
ROUGE-2 F1 Score: 0.0449438202247191
ROUGE-L F1 Score: 0.17582417582417584


#GPT-2

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizergpt = AutoTokenizer.from_pretrained("openai-community/gpt2")
modelgpt = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df=pd.read_csv('/content/samsum-test.csv')
df=df[['dialogue','summary']]
df.head()

,dialogue,summary
0,"Hannah: Hey, do you have Betty's number?\nAman...",Hannah needs Betty's number but Amanda doesn't...
1,Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric:...,Eric and Rob are going to watch a stand-up on ...
2,"Lenny: Babe, can you help me with something?\r...",Lenny can't decide which trousers to buy. Bob ...
3,"Will: hey babe, what do you want for dinner to...",Emma will be home soon and she will let Will k...
4,"Ollie: Hi , are you in Warsaw\r\nJane: yes, ju...",Jane is in Warsaw. Ollie and Jane has a party....


Separating inputs and labels

In [ ]:
inputs=df['dialogue'].tolist()
labels=df['summary'].tolist()

In [ ]:
from sklearn.model_selection import train_test_split
train_inputs,val_inputs,train_labels,val_labels=train_test_split(inputs,labels,test_size=0.2,random_state=42)

In [ ]:
len(train_inputs) , len(val_inputs) , len(train_labels) , len(val_labels)

(655, 164, 655, 164)

In [ ]:
# def tokenize(inputs, labels, tokenizer):
#     # Add a padding token to the tokenizer if it doesn't have one
#     if tokenizergpt.pad_token is None:
#           tokenizergpt.add_special_tokens({'pad_token': '[PAD]'})
#           modelgpt.resize_token_embeddings(len(tokenizergpt))


#     model_input = tokenizer(inputs, max_length=256, padding='max_length', truncation=True)
#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(labels, max_length=60, padding='max_length', truncation=True)

#     model_input['labels'] = labels['input_ids']
#     return model_input

In [ ]:
def tokenize(inputs, labels, tokenizer):
    # Ensure the tokenizer has a pad token
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))

    # Tokenize inputs and labels
    model_input = tokenizer(inputs, max_length=512, padding='max_length', truncation=True, return_tensors="pt")
    labels = tokenizer(labels, max_length=60, padding='max_length', truncation=True, return_tensors="pt")

    model_input['labels'] = labels['input_ids']
    return model_input


In [ ]:
train_tokenized_data=tokenize(train_inputs,train_labels,tokenizergpt)
val_tokenized_data=tokenize(val_inputs,val_labels,tokenizergpt)
train_dataset = Dataset.from_dict(train_tokenized_data)
val_dataset = Dataset.from_dict(val_tokenized_data)


In [ ]:
print(train_tokenized_data.keys())
print(len(train_tokenized_data['input_ids']))
print(len(train_tokenized_data['labels']))
print("Train Dataset Example:", train_dataset[0])
print("Validation Dataset Example:", val_dataset[0])


dict_keys(['input_ids', 'attention_mask', 'labels'])
655
655
Train Dataset Example: {'input_ids': [13787, 25, 5338, 338, 1088, 30, 201, 198, 14295, 25, 3406, 19317, 0, 201, 198, 13787, 25, 1514, 5089, 3511, 3619, 201, 198, 19156, 25, 314, 1101, 407, 287, 3240, 11, 7926, 16133, 0, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257

In [ ]:
from transformers import Trainer, TrainingArguments
# device = "cpu"
# modelgpt.to(device)
training_args = TrainingArguments(
    output_dir='./resultsgpt',
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=10,
)

trainer = Trainer(
    model=modelgpt,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizergpt,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-71-e8fd246c1cfe>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:

trainergpt.train()



RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd

# Load tokenizer and model
tokenizergpt = AutoTokenizer.from_pretrained("openai-community/gpt2")
modelgpt = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

# Add padding token
if tokenizergpt.pad_token is None:
    tokenizergpt.add_special_tokens({'pad_token': '[PAD]'})
    modelgpt.resize_token_embeddings(len(tokenizergpt))

# Load and preprocess dataset
df = pd.read_csv('/content/samsum-test.csv')[['dialogue', 'summary']]
inputs, labels = df['dialogue'].tolist(), df['summary'].tolist()

# Split data
from sklearn.model_selection import train_test_split
train_inputs, val_inputs, train_labels, val_labels = train_test_split(inputs, labels, test_size=0.2, random_state=42)

# Tokenization
def tokenize(inputs, labels, tokenizer):
    model_input = tokenizer(inputs, max_length=512, padding='max_length', truncation=True)
    labels = tokenizer(labels, max_length=60, padding='max_length', truncation=True)
    model_input['labels'] = labels['input_ids']
    return model_input

train_data = tokenize(train_inputs, train_labels, tokenizergpt)
val_data = tokenize(val_inputs, val_labels, tokenizergpt)

train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)

# Training arguments
training_args = TrainingArguments(
    output_dir='./resultsgpt',
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Trainer setup
trainer = Trainer(
    model=modelgpt,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizergpt,
)

# Train
trainer.train()


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-73-be3da3cfd9a7>:49: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Path to the directory where your model is saved
model_directory = "/content/results2/checkpoint-410"  # Update this with your directory path

# Load the saved model and tokenizer from the directory
modelgpt = AutoModelForSeq2SeqLM.from_pretrained(model_directory)
# tokenizer = AutoTokenizer.from_pretrained(model_directory)

# Move the model to the same device (GPU/CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [ ]:
# Assuming your model and tokenizer are already loaded

# Text you want to summarize
text = """Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park
together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him
Hannah: Urgh.. Alright
Hannah: Bye"""

# Tokenize the input text
inputs = tokenizergpt(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Move the input tensors to the same device as the model
inputs = {key: value.to(device) for key, value in inputs.items()}

# Generate the summary
with torch.no_grad():
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=150, length_penalty=2.0, early_stopping=True)

# Decode the generated summary
summary = tokenizergpt.decode(summary_ids[0], skip_special_tokens=True)

# Print the summary
print("Generated Summary:", summary)


Generated Summary: #!annah and Betty's number. Betty's as at the park. Betty will be shy, he's very nice. Betty doesn't know him well. Betty is going to him. Betty has to the texted him.  igunfights Amelia Rik Melanie insepar Cannes dunno learns Giovanni Judah guaranteeing" composure Tad debian paternity � 7000 mum rodent migraine Explan 9000Terry caches dissidents retrieving David gazing but Nathan orphans#


In [ ]:
import evaluate

# Load the Rouge metric
rouge = evaluate.load("rouge")
reference_summary=text
generated_summary=summary
results = rouge.compute(predictions=[generated_summary], references=[reference_summary])
print("Rouge Score:", results)
print("ROUGE-1 F1 Score:", results["rouge1"])
print("ROUGE-2 F1 Score:", results["rouge2"])
print("ROUGE-L F1 Score:", results["rougeL"])

Rouge Score: {'rouge1': 0.2714285714285714, 'rouge2': 0.18840579710144928, 'rougeL': 0.2, 'rougeLsum': 0.2714285714285714}
ROUGE-1 F1 Score: 0.2714285714285714
ROUGE-2 F1 Score: 0.18840579710144928
ROUGE-L F1 Score: 0.2


/bin/bash: line 1: zip-r: command not found


#bert

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizerbt = AutoTokenizer.from_pretrained("ntkhoi/BERT-Text-Summarization")
modelbt = AutoModelForSeq2SeqLM.from_pretrained("ntkhoi/BERT-Text-Summarization")

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/247M [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "google/bert_uncased_L-4_H-512_A-8",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "return_dict": false,
  "transformers_version": "4.46.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

Config of the decoder: <class 'transformers.models.bert.modeling_bert.BertLMHeadModel'> is overwritten by shared decoder config: BertConfig {
  "_name_or_path": "google/bert_uncased_L-4_H-512_A-8",
  "add_cross_attention": true,
  "attenti

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df=pd.read_csv('/content/samsum-test.csv')
df=df[['dialogue','summary']]
df.head()

,dialogue,summary
0,"Hannah: Hey, do you have Betty's number?\nAman...",Hannah needs Betty's number but Amanda doesn't...
1,Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric:...,Eric and Rob are going to watch a stand-up on ...
2,"Lenny: Babe, can you help me with something?\r...",Lenny can't decide which trousers to buy. Bob ...
3,"Will: hey babe, what do you want for dinner to...",Emma will be home soon and she will let Will k...
4,"Ollie: Hi , are you in Warsaw\r\nJane: yes, ju...",Jane is in Warsaw. Ollie and Jane has a party....


Separating inputs and labels

In [ ]:
inputs=df['dialogue'].tolist()
labels=df['summary'].tolist()

In [ ]:
from sklearn.model_selection import train_test_split
train_inputs,val_inputs,train_labels,val_labels=train_test_split(inputs,labels,test_size=0.2,random_state=42)

In [ ]:
len(train_inputs) , len(val_inputs) , len(train_labels) , len(val_labels)

(655, 164, 655, 164)

In [ ]:
def tokenize(inputs, labels, tokenizer):
    # Add a padding token to the tokenizer if it doesn't have one
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    model_input = tokenizer(inputs, max_length=512, padding='max_length', truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(labels, max_length=60, padding='max_length', truncation=True)

    model_input['labels'] = labels['input_ids']
    return model_input

In [ ]:
from datasets import Dataset

In [ ]:
train_tokenized_data=tokenize(train_inputs,train_labels,tokenizerbt)
val_tokenized_data=tokenize(val_inputs,val_labels,tokenizerbt)
train_dataset = Dataset.from_dict(train_tokenized_data)
val_dataset = Dataset.from_dict(val_tokenized_data)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir='./resultsbert',
    evaluation_strategy="epoch",  # Evaluate after every epoch
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=10,  # Log every 10 steps
)
trainer = Seq2SeqTrainer(
    model=modelbt,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # Ensure you provide this for evaluation
    tokenizer=tokenizerbt,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-14-40d004b6cfa7>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()



wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sheikhmudassar1942003 (sheikhmudassar1942003-self). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:629: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:649: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Epoch,Training Loss,Validation Loss
1,1.248400,1.071849
2,0.959300,1.032734
3,0.852000,1.025191
4,0.700400,1.033869
5,0.659800,1.046327
6,0.582200,1.051906
7,0.571400,1.059050
8,0.528300,1.065700
9,0.476700,1.071275
10,0.460900,1.072608


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:629: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:649: FutureWarning: Version v4.12.0 introduce

TrainOutput(global_step=410, training_loss=0.8120748147731874, metrics={'train_runtime': 183.9318, 'train_samples_per_second': 35.611, 'train_steps_per_second': 2.229, 'total_flos': 603337523712000.0, 'train_loss': 0.8120748147731874, 'epoch': 10.0})

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Path to the directory where your model is saved
model_directory = "/content/resultsbert/checkpoint-410"  # Update this with your directory path

# Load the saved model and tokenizer from the directory
modelbt = AutoModelForSeq2SeqLM.from_pretrained(model_directory)
# tokenizer = AutoTokenizer.from_pretrained(model_directory)

# Move the model to the same device (GPU/CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
modelbt.to(device)


Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "google/bert_uncased_L-4_H-512_A-8",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "return_dict": false,
  "transformers_version": "4.46.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

Config of the decoder: <class 'transformers.models.bert.modeling_bert.BertLMHeadModel'> is overwritten by shared decoder config: BertConfig {
  "_name_or_path": "google/bert_uncased_L-4_H-512_A-8",
  "add_cross_attention": true,
  "attenti

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 512, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12, elem

In [ ]:
# Assuming your model and tokenizer are already loaded

# Text you want to summarize
text = """Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park
together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him
Hannah: Urgh.. Alright
Hannah: Bye"""

# Tokenize the input text
inputs = tokenizerbt(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Move the input tensors to the same device as the model
inputs = {key: value.to(device) for key, value in inputs.items()}

# Generate the summary
with torch.no_grad():
    summary_ids = modelbt.generate(inputs['input_ids'], num_beams=4, max_length=150, length_penalty=2.0, early_stopping=True)

# Decode the generated summary
summary = tokenizerbt.decode(summary_ids[0], skip_special_tokens=True)

# Print the summary
print("Generated Summary:", summary)


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Generated Summary: hannah needs betty's number. she doesn't know him well, but he's very nicer. hannah is nicer, but she's too busy to text him. hannah will send hannah to the park with a text from larry. they are at the park together.


In [ ]:
import evaluate

# Load the Rouge metric
rouge = evaluate.load("rouge")
reference_summary=text
generated_summary=summary
results = rouge.compute(predictions=[generated_summary], references=[reference_summary])
print("Rouge Score:", results)
print("ROUGE-1 F1 Score:", results["rouge1"])
print("ROUGE-2 F1 Score:", results["rouge2"])
print("ROUGE-L F1 Score:", results["rougeL"])

Rouge Score: {'rouge1': 0.35000000000000003, 'rouge2': 0.20338983050847456, 'rougeL': 0.2666666666666667, 'rougeLsum': 0.35000000000000003}
ROUGE-1 F1 Score: 0.35000000000000003
ROUGE-2 F1 Score: 0.20338983050847456
ROUGE-L F1 Score: 0.2666666666666667
